Author: Claudia Gusti
Date: 12/3/22

Approaches to Multi Class Food Image Classification



# Download and extract Food 101 dataset

In [ ]:
from __future__ import absolute_import, division, print_function

#Necessay tensorflow imports
import tensorflow as tf

import tensorflow.keras.backend as K
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras import regularizers
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
from tensorflow.keras.optimizers import SGD, RMSprop
from tensorflow.keras.regularizers import l2
from tensorflow.keras import callbacks
from tensorflow.keras.callbacks import TensorBoard, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications import VGG16, VGG19


from tensorflow import keras
from tensorflow.keras import models
from tensorflow.keras.applications.inception_v3 import preprocess_input

# For image processing, visualizing images and plotting graphs
import cv2
import os
import random
import collections
from collections import defaultdict

from shutil import copy
from shutil import copytree, rmtree
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as img
%matplotlib inline

In [ ]:
#Make sure GPU has enough space
from GPUtil import showUtilization as gpu_usage

import gc
import torch

gc.collect()
torch.cuda.empty_cache()

gpu_usage()

In [ ]:
# Check TF version and whether GPU is enabled
print(tf.__version__)
print(tf.test.gpu_device_name())

'''Expected output: 
2.2.0
/device:GPU:0'''

Next, let's download and extract [Food-101](https://data.vision.ee.ethz.ch/cvl/datasets_extra/food-101/) dataset

In [ ]:
# Helper function to download and extract data and extract

def get_data():
  if "food-101" in os.listdir():
    print("Dataset already exists")
  else:
    tf.keras.utils.get_file(
    'food-101.tar.gz',
    'http://data.vision.ee.ethz.ch/cvl/food-101.tar.gz',
    cache_subdir='./content/',
    extract=True,
    archive_format='tar',
    cache_dir=None
    )
    print("Dataset downloaded and extracted successfully!")

In [ ]:
# Call the helper function
get_data() 

# Analysing data and visualizing images

In [ ]:
#Lets have a look into the directory
os.listdir('../.keras/content/food-101/')

This dataset has 101 different classes of food with 1000 images for each class.
This 1000 images are divided into 750 training samples and 250 test samples

Next we list all the classes/labels of the dataset

In [ ]:
#List of all classes
foods_sorted = sorted(os.listdir('../.keras/content/food-101/images'))
foods_sorted

In [ ]:
#The meta file contains labels for train and test in txt format as well 101 classes
os.listdir('../.keras/content/food-101/meta')

Now let's visualize a randomly selected image from every class

In [ ]:
# Visualize the data, showing one image per class from 101 classes
rows = 17
cols = 6
fig, ax = plt.subplots(rows, cols, figsize=(50,50))
fig.suptitle("Showing one random image from each class", y=1.05, fontsize=60)
data_dir = "../.keras/content/food-101/images"
foods_sorted = sorted(os.listdir(data_dir))
food_id = 0
for i in range(rows):
  for j in range(cols):
    try:
      food_selected = foods_sorted[food_id] 
      food_id += 1
    except:
      break
    food_selected_images = os.listdir(os.path.join(data_dir,food_selected)) # returns the list of all files present in each food category
    food_selected_random = np.random.choice(food_selected_images) # picks one food item from the list as choice, takes a list and returns one random item
    img = plt.imread(os.path.join(data_dir,food_selected, food_selected_random))
    ax[i][j].imshow(img)
    ax[i][j].set_title(food_selected, pad = 20,fontsize=40)
    
plt.setp(ax, xticks=[],yticks=[])
plt.tight_layout()

# Splitting the data into train and test set

In [ ]:
# Helper method to split dataset into train and test folders
from shutil import copy
def prepare_data(filepath, src, dest):
  classes_images = defaultdict(list)
  with open(filepath, 'r') as txt:
      paths = [read.strip() for read in txt.readlines()]
      for p in paths:
        food = p.split('/')
        classes_images[food[0]].append(food[1] + '.jpg')

  for food in classes_images.keys():
    print("\nCopying images into ",food)
    if not os.path.exists(os.path.join(dest,food)):
      os.makedirs(os.path.join(dest,food))
    for i in classes_images[food]:
      copy(os.path.join(src,food,i), os.path.join(dest,food,i))
  print("Copying Done!")

In [ ]:
# Prepare train dataset by copying images from food-101/images to food-101/train using the file train.txt
print("Creating train data...")
prepare_data('../.keras/content/food-101/meta/train.txt', '../.keras/content/food-101/images', '../.keras/content/food-101/train')

In [ ]:
# Prepare test data by copying images from food-101/images to food-101/test using the file test.txt
print("Creating test data...")
prepare_data('../.keras/content/food-101/meta/test.txt', '../.keras/content/food-101/images', '../.keras/content/food-101/test')

# Create a subset of data to test some SOTA models



*   Experimenting different architectures on the complete dataset would take a lot of time and computation power. 
*   So instead we create a subset of 4 classes and test a couple of architectures for evaluting the performance


In [ ]:
# Helper method to create train_mini and test_mini data samples
from shutil import copytree, rmtree
def dataset_mini(food_list, src, dest):
  if not os.path.exists(dest):
    os.makedirs(dest)   #Make a directory if it does not exists
  for food_item in food_list :
    print("Copying images into",food_item)
    copytree(os.path.join(src,food_item), os.path.join(dest,food_item))

In [ ]:
# picking 4 random food items and generating separate data folders for the same
food_list = ['apple_pie','cannoli','dumplings', 'miso_soup']  #You can choose any food items
src_train = '../.keras/content/food-101/train'
dest_train = '../.keras/content/food-101/train_mini'
src_test = '../.keras/content/food-101/test'
dest_test = '../.keras/content/food-101/test_mini'

In [ ]:
#Create subset for training data 
print("Creating train data folder with new classes")
dataset_mini(food_list, src_train, dest_train)

In [ ]:
#Create subset for test data
print("Creating test data folder with new classes")
dataset_mini(food_list, src_test, dest_test)

All the below models have been referred from  [table](https://keras.io/api/applications/) to get an insight for Top-1 and Top-5 accuracy of various SOTA models

# Testing some State-Of-The-Art model on mini dataset


## Testing VGG16 model

In [ ]:
# Declare some variables

n_classes = 4   #num of output classes
img_width, img_height = 224, 224   #Default image size for VGG16 model
train_data_dir = '../.keras/content/food-101/train_mini'
validation_data_dir = '../.keras/content/food-101/test_mini'
batch_size = 16

In [ ]:
# Perform data augmentation using Image data generator
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')  #Since it's a multiclass classification so class_mode = 'categorical'

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')


In [ ]:
VGG16 = VGG16(weights='imagenet', include_top=False)
x = VGG16.output
x = GlobalAveragePooling2D()(x)
x = Dense(128,activation='relu')(x)
x = Dropout(0.2)(x)    #Dropout to prevent overfitting

                              #L2 regularization to prevent overfitting
predictions = Dense(n_classes ,kernel_regularizer=regularizers.l2(0.005), activation='softmax')(x)

model = Model(inputs=VGG16.input, outputs=predictions)

#Compile the model            #Learning rate = 0.001
model.compile(optimizer=SGD(lr=0.001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])


#fit the model
history_VGG16 = model.fit_generator(train_generator,
                    validation_data=validation_generator,
                    epochs=10,
                    verbose=1)

## Testing InceptionV3 model

In [ ]:
# Declare some variables

n_classes = 4   #num of output classes
img_width, img_height = 299, 299   #Default image size for Inception and Xception model
train_data_dir = '../.keras/content/food-101/train_mini'
validation_data_dir = '../.keras/content/food-101/test_mini'
batch_size = 16

In [ ]:
# Perform data augmentation using Image data generator
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')  #Since it's a multiclass classification so class_mode = 'categorical'

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')


In [ ]:
inception = InceptionV3(weights='imagenet', include_top=False)
#add a global spatial average pooling layer
x = inception.output
x = GlobalAveragePooling2D()(x)
#add a fully connected layer
x = Dense(128,activation='relu')(x)
x = Dropout(0.2)(x)    #Dropout to prevent overfitting


#add a logistic layer - this is the output layer that will predict n number of class 
#L2 regularization to prevent overfitting
predictions = Dense(n_classes ,kernel_regularizer=regularizers.l2(0.005), activation='softmax')(x)

model = Model(inputs=inception.input, outputs=predictions)

#Compile the model            #Learning rate = 0.001
model.compile(optimizer=SGD(lr=0.001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])


#fit the model
history_inception = model.fit_generator(train_generator,
                    validation_data=validation_generator,
                    epochs=10,
                    verbose=1)

## Testing Xception model

In [ ]:
xception = Xception(weights='imagenet', include_top=False)
x = xception.output
x = GlobalAveragePooling2D()(x)
x = Dense(128,activation='relu')(x)
x = Dropout(0.2)(x)    #Dropout to prevent overfitting

                           #L2 regularization to prevent overfitting 
predictions = Dense(n_classes ,kernel_regularizer=regularizers.l2(0.005), activation='softmax')(x)

model = Model(inputs=xception.input, outputs=predictions)

#Compile the model     #Learning rate = 0.001
model.compile(optimizer=SGD(lr=0.001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])




history_xception = model.fit_generator(train_generator,
                    validation_data=validation_generator,
                    epochs=10,
                    verbose=1)

# Plotting the results for all the 3 models

In [ ]:
import matplotlib.pyplot as plt
#Function to plot Accuracy
def plot_accuracy(title):
    plt.title(title)
    plt.plot(history_inception.history['val_accuracy'])
    plt.plot(history_xception.history['val_accuracy'])
    plt.plot(history_VGG16.history['val_accuracy'])
    plt.ylabel('Validation accuracy')
    plt.xlabel('epoch')
    plt.legend(['InceptionV3', 'Xception', 'VGG16'], loc='best')
    plt.show()

#Function to plot Loss
def plot_loss(title):
    plt.title(title)
    plt.plot(history_inception.history['val_loss'])
    plt.plot(history_xception.history['val_loss'])
    plt.plot(history_VGG16.history['val_loss'])
    plt.ylabel('Validation Loss')
    plt.xlabel('epoch')
    plt.legend(['InceptionV3', 'Xception', 'VGG16'], loc='best')
    plt.show()


plot_accuracy('Accuracy comparison of Models')
plot_loss('Loss comparison of Models')

##Having a closer look on accuracy and loss

In [ ]:

plt.plot(history_inception.history['val_accuracy'][4:])
plt.plot(history_xception.history['val_accuracy'][4:])
plt.plot(history_VGG16.history['val_accuracy'][4:])
plt.ylabel('Validation accuracy')
plt.xlabel('epoch')
plt.legend(['InceptionV3', 'Xception', 'VGG16'], loc='best')
plt.show()

In [ ]:
plt.plot(history_inception.history['val_loss'][4:])
plt.plot(history_xception.history['val_loss'][4:])
plt.plot(history_VGG16.history['val_loss'][4:])
plt.ylabel('Validation accuracy')
plt.xlabel('epoch')
plt.legend(['InceptionV3', 'Xception', 'ResNet50'], loc='best')
plt.show()

From the above analysis, Inception modelV3 gives better convergence as compared to Inception and ResNet50 for Food-101 dataset.

# Fine Tuning InceptionV3

#Fine tuning process for InceptionV3 model all 101 classes
 
Fine tuning methodology: 
1. Freezing all layers learning a classifier on top of it - similar to transfer learning (and using data augmentation)
2. Training the last 3 convolutional layers (with data augmentation)


In [ ]:
# Declare some constants

n_classes = 101   # total num of output classes
img_width, img_height = 299, 299   #Default image size for InceptionV3 model
batch_size = 32

train_data_dir = '../.keras/content/food-101/train'   #Directory for train dataset
validation_data_dir = '../.keras/content/food-101/test'     #Directory for test dataset

Now we use [Image Data Generator](https://keras.io/api/preprocessing/image/) for performing data augmentation

In [ ]:
# Perform advance data augmentation using Image data generator
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    rotation_range = 40,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')  #Since it's a multiclass classification so class_mode = 'categorical'

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')


##Loading InceptionV3 model

Let's load the InceptionV3 model and attach Fully connected layers on top of it.

In [ ]:
inceptionV3 = InceptionV3(weights='imagenet', include_top=False, input_shape=(299,299,3))
x = inceptionV3.output
x = GlobalAveragePooling2D()(x)

#I have just used 128 hidden units due to restricted Colab GPU runtime. 
#For such huge dataset, 1024 or 512 hidden units should be ideally preferred.
x = Dense(128,activation='relu')(x)  
x = Dropout(0.4)(x)    #Dropout to prevent overfitting

#Freeze all the layers of the xception model
for layer in inceptionV3.layers:
  layer.trainable = False

                           #L2 regularization to prevent overfitting 
predictions = Dense(n_classes ,kernel_regularizer=regularizers.l2(0.0001), activation='softmax')(x)

model = Model(inputs=inceptionV3.input, outputs=predictions)




In [ ]:
#Visualize the model
model.summary()


##Compile the model and define custom callbacks

In [ ]:
#Compile the model            #Learning rate = 0.001
model.compile(optimizer=SGD(lr=0.001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

#Callback for model saving and reducing learning rate on plateau
callbacks_list = [callbacks.ModelCheckpoint(
        filepath = './InceptionV3-model.h5',
        monitor = 'val_loss',
        save_best_only = True),
        callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.1,
            patience=2,
            mode='min',
            min_lr=1e-8)]



In [ ]:
#Fit the model
history = model.fit_generator(train_generator,
                    validation_data=validation_generator,
                    epochs=10,
                    verbose=1,
                    callbacks = callbacks_list)

In [ ]:
#Utility function for plotting of the model results
def visualize_results(history):
    # Plot the accuracy and loss curves
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
 
    epochs = range(len(acc))
 
    plt.plot(epochs, acc, 'b', label='Training acc')
    plt.plot(epochs, val_acc, 'r', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()
 
    plt.figure()
 
    plt.plot(epochs, loss, 'b', label='Training loss')
    plt.plot(epochs, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()
 
    plt.show()




In [ ]:
visualize_results(history)

In [ ]:
#utility function for obtaining errors

def obtain_errors(val_generator, predictions):
    # Get the filenames from the generator
    fnames = validation_generator.filenames
 
    # Get the ground truth from generator
    ground_truth = validation_generator.classes
 
    # Get the dictionary of classes
    label2index = validation_generator.class_indices
 
    # Obtain the list of the classes
    idx2label = list(label2index.keys())
    print("The list of classes: ", idx2label)
 
    # Get the class index
    predicted_classes = np.argmax(predictions, axis=1)
 
    errors = np.where(predicted_classes != ground_truth)[0]
    print("Number of errors = {}/{}".format(len(errors),validation_generator.samples))
     
    return idx2label, errors, fnames
 

 

In [ ]:
def show_errors(idx2label, errors, predictions, fnames):
    # Show the errors
    for i in range(len(errors)):
        pred_class = np.argmax(predictions[errors[i]])
        pred_label = idx2label[pred_class]
 
        title = 'Original label:{}, Prediction :{}, confidence : {:.3f}'.format(
            fnames[errors[i]].split('/')[0],
            pred_label,
            predictions[errors[i]][pred_class])
 
        original = load_img('{}/{}'.format(validation_data_dir,fnames[errors[i]]))
        plt.figure(figsize=[7,7])
        plt.axis('off')
        plt.title(title)
        plt.imshow(original)
        plt.show()

In [ ]:
#pipeline for error analysis

# example of loading an image with the Keras API
from keras.preprocessing.image import load_img


predictions = model.predict(validation_generator, steps=validation_generator.samples/validation_generator.batch_size,verbose=1)
 
# Run the function to get the list of classes and errors
idx2label, errors, fnames = obtain_errors(validation_generator, predictions)
 


In [ ]:
# Run the function to illustrate the error cases
show_errors(idx2label, errors, predictions, fnames)

## Training the last convolutional networks 

Let's try to fine-tune the model

In [ ]:
len(model.layers)

In [ ]:
Lets choose the layers which are updated by training

In [ ]:
# Freeze layers till 
for layer in model.layers[:279]:
  layer.trainable =  False

#Train weights from 
for layer in model.layers[279:]:
  layer.trainable = True


In [ ]:
model.summary()

In [ ]:
#Compile the model            #Learning rate = 0.0001
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

#Callback for model saving and reducing learning rate on plateau
callbacks_list = [callbacks.ModelCheckpoint(
        filepath = './InceptionV3-model.h5',
        monitor = 'val_accuracy',
        save_best_only = True),
        callbacks.ReduceLROnPlateau(
            monitor='val_accuracy',
            factor=0.1,
            patience=2,
            mode='max',
            min_delta=0.002,
            min_lr=1e-8),
        callbacks.EarlyStopping(
    monitor='val_accuracy', min_delta=0.001, patience=3, verbose=1, mode='max',
    baseline=None)]


In [ ]:
#Fit the model
history = model.fit_generator(train_generator,
                    validation_data=validation_generator,
                    epochs=10,
                    verbose=1,
                    callbacks = callbacks_list)

In [ ]:
def visualize_results(history):
    # Plot the accuracy and loss curves
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
 
    epochs = range(len(acc))
 
    plt.plot(epochs, acc, 'b', label='Training acc')
    plt.plot(epochs, val_acc, 'r', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()
 
    plt.figure()
 
    plt.plot(epochs, loss, 'b', label='Training loss')
    plt.plot(epochs, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()
 
    plt.show()


In [ ]:
predictions = model.predict(validation_generator, steps=validation_generator.samples/validation_generator.batch_size,verbose=1)
 
# Run the function to get the list of classes and errors
idx2label, errors, fnames = obtain_errors(validation_generator, predictions)
 
# Run the function to illustrate the error cases
show_errors(idx2label, errors, predictions, fnames)

# Conclusions

In [ ]:
model.evaluate(validation_generator)